# Load and Inspect Kickstarter Data
This notebook demonstrates how to load a CSV dataset using pandas and perform basic inspection.

In [ ]:
# Import the pandas library, our main tool for working with data
import pandas as pd

# Define the filename for our dataset
file_path = 'kickstarter.csv'

# Load the CSV file into a pandas DataFrame
# We use a try-except block to handle a potential FileNotFoundError
try:
    df = pd.read_csv(file_path)
    print("✅ Dataset loaded successfully!")
except FileNotFoundError:
    print(f"❌ Error: The file '{file_path}' was not found.")
    print("Please make sure the dataset is in the same directory as your notebook, or provide the correct path.")

In [ ]:
# If the dataframe was loaded, let's inspect it
if 'df' in locals():
    # Display the first 5 rows to get a feel for the data
    print("\n--- First 5 Rows ---")
    display(df.head())

    # Display a concise summary of the DataFrame
    print("\n--- Dataframe Info ---")
    df.info()

    # Display descriptive statistics for numerical columns
    print("\n--- Descriptive Statistics ---")
    display(df.describe())